In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy tensorflow keras

In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB0

# Define directory locations and other parameters
train_data_dir = '/content/drive/MyDrive/Dataset/Train'
validation_data_dir = '/content/drive/MyDrive/Dataset/Test'
num_classes = 3  # Number of classes (2 for binary classification)
batch_size = 32
epochs = 10

# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model and specify metrics
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define ImageDataGenerator for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

# Train the model
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

# Save the trained model
model.save("/content/Model/ImageForgeryDetection.h5")

Found 180 images belonging to 3 classes.
Found 180 images belonging to 3 classes.
Epoch 1/10
6/6 [==============================] - 163s 26s/step - loss: 0.4517 - accuracy: 0.7611 - val_loss: 0.9226 - val_accuracy: 0.8000
Epoch 2/10
6/6 [==============================] - 63s 11s/step - loss: 0.0392 - accuracy: 0.9833 - val_loss: 0.8809 - val_accuracy: 0.8500
Epoch 3/10
6/6 [==============================] - 64s 11s/step - loss: 0.0614 - accuracy: 0.9833 - val_loss: 1.0500 - val_accuracy: 0.8611
Epoch 4/10
6/6 [==============================] - 54s 9s/step - loss: 0.0671 - accuracy: 0.9778 - val_loss: 1.0681 - val_accuracy: 0.8722
Epoch 5/10
6/6 [==============================] - 54s 9s/step - loss: 0.1395 - accuracy: 0.9722 - val_loss: 0.5748 - val_accuracy: 0.9333
Epoch 6/10
6/6 [==============================] - 54s 9s/step - loss: 0.0281 - accuracy: 0.9889 - val_loss: 0.4244 - val_accuracy: 0.9722
Epoch 7/10
6/6 [==============================] - 63s 11s/step - loss: 0.0166 - accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
# Define the custom layer
class FixedDropout(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(FixedDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

    def get_config(self):
        config = super(FixedDropout, self).get_config()
        config.update({'rate': self.rate})
        return config

# Load the model within the custom object scope
with tf.keras.utils.custom_object_scope({'FixedDropout': FixedDropout}):
    model = load_model('/content/Model/ImageForgeryDetection.h5')


# Function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Adjust target size as per your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make prediction
def predict_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:  # Assuming the model returns a probability for class 1
        return "Forged"
    else:
        return "Authentic"

# Path to the image you want to predict
image_path = '/content/InputData/Tp_D_CND_S_N_txt00028_txt00006_10848.jpg'

# Perform prediction
prediction = predict_image(image_path)
print("Prediction:", prediction)



1/1 [==============================] - 2s 2s/step
Prediction: Authentic
